## Введение в анализ данных, ИАД-2

## НИУ ВШЭ, 2018-19 учебный год

### Домашнее задание №3

Задание выполнил(а): _Вертеева Алена Сергеевна_

### Общая информация

__Дата выдачи:__ 06.05.2019

__Дедлайн:__ 23:59 24.05.2019

### Оценивание и штрафы

Оценка за ДЗ вычисляется по следующей формуле:

$$
\text{points} \times 10 / 9,
$$

где points — количество баллов за обязательную часть, которое вы набрали. Максимальное число баллов, которое можно получить за обязательную часть — 9, а максимальное число дополнительных баллов, которые пойдут в бонус — 1.5.

За сдачу задания позже срока на итоговую оценку за задание накладывается штраф в размере 1 балл в день, но получить отрицательную оценку нельзя.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

### Формат сдачи

Загрузка файлов с решениями происходит в системе [Anytask](https://anytask.org/).

Инвайт для группы ИАД-2: zCH4F32

In [76]:
import numpy as np
from sklearn.datasets import load_breast_cancer, load_boston
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

# Random Forest своими руками (4)

Реализуйте алгоритм случайного леса. В соответствии с построением алгоритма, ваша реализация должна сочетать в себе метод бэггинга и метод случайных подмножеств. Также она должна работать для двух типов задач: как для задачи регрессии, так и для задачи классификации.

*Примечание. Вы можете использовать `DecisionTree` из `sklearn`.*

In [77]:
class RandomForest:
    def __init__(self, n_estimators, max_depth, max_features='sqrt', task_type='regression'):
        """
        PARAMETERS:
        n_estimators - number of trees in the ensemble
        max_depth - maximum depth of a tree
        max_features - maximum features for each split in DecisionTree from sklearn
        task_type - 'regression' or 'classification'
        """
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        self.task_type = task_type
        self.X_train = None
        self.y_train = None
        self.tree = None
        
    def fit(self, X_train, y_train):
        """
        INPUT:
        X_train - np.array of shape (k, d)
        y_train - np.array of shape (k,)
        """
        self.X_train = X_train
        self.y_train = y_train
        
        indecies = np.random.randint(self.X_train.shape[0], size=int(self.X_train.shape[0] * 0.9))
        
        if self.task_type == 'regression':
            self.tree = DecisionTreeRegressor(max_depth=self.max_depth,
                                         max_features=self.max_features)
        else:
            self.tree = DecisionTreeClassifier(max_depth=self.max_depth, 
                                          max_features=self.max_features)
        
        self.tree.fit(self.X_train[indecies], self.y_train[indecies])
        
        return self
        
    def predict(self, X_test):
        """
        INPUT:
        X_test - np.array of shape (m, d)
        
        OUTPUT:
        y_pred - np.array of shape (m,)
        """
        mas = np.zeros(X_test.shape[0])
        if self.task_type == 'regression':
            for i in range(self.n_estimators):
                self.fit(self.X_train, self.y_train)
                mas += self.tree.predict(X_test)
            mas /= self.n_estimators
        else:
            for i in range(self.n_estimators):
                self.fit(self.X_train, self.y_train)
                pr = self.tree.predict(X_test)
                for j in range(pr.shape[0]):
                    if pr[j] == 0:
                        pr[j] = -1
                mas += pr
            for i in range(mas.shape[0]):
                if mas[i] <= 0:
                    mas[i] = 0
                else:
                    mas[i] = 1
        y_pred = mas
        return y_pred

Проверьте вашу реализацию на задаче регрессии. Подберите оптимальные гиперпараметры, чтобы получить MSE < 10.25 **(не меняйте параметры сида)**.

In [78]:
boston = load_boston()
X = boston.data
y = boston.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

In [79]:
X_test.shape

(127, 13)

In [80]:
np.random.seed(19052019)

rf = RandomForest(500, 10)
rf.fit(X_train,y_train)
al = rf.predict(X_test)

In [81]:
print('Mean Squared Error: ', mean_squared_error(y_test, al))

Mean Squared Error:  10.071812315930092


Проверьте вашу реализацию на задаче классификации. Подберите оптимальные гиперпараметры, чтобы получить accuracy > 0.83 **(не меняйте параметры сида)**.

In [82]:
breast_cancer = load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

In [83]:
X_test.shape

(143, 30)

In [84]:
np.random.seed(19052019)

rf = RandomForest(500, 10, task_type='classification')
rf.fit(X_train,y_train)
al = rf.predict(X_test)

In [85]:
print('Accuracy score: ', accuracy_score(y_test, al))

Accuracy score:  0.9300699300699301


# Градиентный бустинг своими руками (4)

Реализуйте алгоритм градиентного бустинга для регрессии. Напомним основные формулы.

Обозначим текущую композицию на $N-1$ шаге за $a_{N - 1}(x_i)$. Следующий базовый алгоритм $b_N(x_i)$ обучается на ответах $-\frac{\partial L(y_i, z)}{\partial z}\Bigl|_{z = a_{N - 1}(x_i)}$, где $L(y_i, z)$ — значение функции потерь на объекте при правильном ответе $y_i$ и предсказании $z$. Композиция на следующем шаге получается следующим образом:

$$
a_N(x_i) = a_{N-1}(x_i) + \nu\gamma_Nb_N(x_i)
$$

Здесь $\nu$ — гиперпараметр learning rate, $\gamma_N$ — оптимальный вес, настраиваемый на каждом шаге алгоритма, который можно найти по следующей формуле (обратите внимание на отсутствие $\nu$):

$$
\gamma_N = \mathrm{arg}\min_\gamma \frac{1}{\ell}\sum\limits_{i=1}^{\ell}L\left(y_i, a_{N - 1}(x_i) + \gamma b_N(x_i)\right)
$$

Можете принять $\gamma_N = 1$ для каждого $N$. Однако вы можете получить дополнительные 0.5 баллов за реализацию нахождения оптимального $\gamma_N$ на каждом шаге.

В качестве функции потерь возьмите MSE.

*Примечание. Вы можете использовать `DecisionTree` из `sklearn` и методы оптимизации из различных библиотек.*

In [86]:
class GradientBoosting:
    def __init__(self, n_estimators, max_depth, learning_rate=0.1):
        """
        PARAMETERS:
        n_estimators - number of trees in the ensemble
        max_depth - maximum depth of a tree
        learning_rate - coefficient by which new algorithm result is multiplied
        """
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.learning_rate = learning_rate
        self.X_train = None
        self.y_train = None
        self.tree = None
        self.pred = None
        
    def fit(self, X_train, y_train):
        """
        INPUT:
        X_train - np.array of shape (k, d)
        y_train - np.array of shape (k,)
        """
        self.X_train = X_train
        self.y_train = y_train
        self.tree = DecisionTreeRegressor(max_depth=self.max_depth)
        self.tree.fit(self.X_train, self.y_train)
        self.pred = self.tree.predict(self.X_train)

        return self
        
    def predict(self, X_test):
        """
        INPUT:
        X_test - np.array of shape (m, d)
        
        OUTPUT:
        y_pred - np.array of shape (m,)
        """
        y_pred = np.zeros(X_test.shape[0])
        for i in range(self.n_estimators - 1):
            a = self.X_train
            b = self.y_train
            c = self.pred
            y_pred += self.tree.predict(X_test) * self.learning_rate
            self.fit(a, b - self.learning_rate * c)
            
        return y_pred

Проверьте вашу реализацию. Подберите оптимальные гиперпараметры, чтобы получить MSE < 10 как в обычном случае, так и при нахождении оптимального шага **(не меняйте параметры сида)**.

In [87]:
X = boston.data
y = boston.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

In [88]:
np.random.seed(19052019)

gb = GradientBoosting(1000, 6, 0.2)
gb.fit(X_train,y_train)
qwe = gb.predict(X_test)

In [89]:
print('Mean Squared Error: ', mean_squared_error(y_test, qwe))

Mean Squared Error:  9.805276850475614


Для удобства проверяющего укажите, пожалуйста, реализовали ли вы нахождение оптимального шага?

- Да / Нет

# Крестики-нолики (1)

In [90]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

В этом задании вы поработаете с [данными](https://datahub.io/machine-learning/tic-tac-toe-endgame), состоящими из конечных позиций в игру крестики-нолики. Признаками являются клетки, их значениями — то, что изображено в данной клетке на момент окончания игры (крестик, нолик или пусто). Стоит задача бинарной классификации — определить, выиграли крестики (класс `True`) или нолики (класс `False`). Напомним, что одна из сторон выигрывает в том случае, если либо одна из диагоналей, либо одна из горизонталей, либо одна из вертикалей заполнена соответствующими символами.

In [91]:
data = pd.read_csv('tic-tac-toe.csv')
X = data.drop('class', axis=1)
y = data['class'].astype(int)
data.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [92]:
X = X.applymap(lambda y: 1 if y == 'x' else -1 if y == 'o' else 0)
X.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR
0,1,1,1,1,-1,-1,1,-1,-1
1,1,1,1,1,-1,-1,-1,1,-1
2,1,1,1,1,-1,-1,-1,-1,1
3,1,1,1,1,-1,-1,-1,0,0
4,1,1,1,1,-1,-1,0,-1,0


Применим метод случайного леса к данным и посмотрим на среднее значение accuracy на кросс-валидации.

In [93]:
rf = RandomForestClassifier(n_estimators=500, max_depth=10, random_state=19052019)
score = cross_val_score(rf, X, y, cv=5, scoring='accuracy').mean()
print(score)

0.8917722942607311


Задача: улучшите данное значение до >0.945 с помощью Feature Engineering.

In [94]:
tr = DecisionTreeClassifier()
tr.fit(X,y)
accuracy_score(y, tr.predict(X))

1.0

**Примечание 1. Нельзя менять гиперпараметры алгоритма! Можно только делать что-нибудь с признаковым пространством.**

**Примечание 2. 1 балл ставится за банальную проверку, что кто-то выиграл - что одна из 8 линий заполнена. А еще 1 балл (БОНУСНЫЙ) можно заработать чем-то более хитрым :) Вспомните методы извлечения признаков, которые мы проходили, можете сжимать размерность, трансформировать признаки, в общем, делать что угодно. Подсказка (которой необязательно следовать) - подумайте, что можно сделать с признаковым пространством, чтобы с помощью него можно было также провести своего рода "проверку на выигрыш", но только без прописываний 8 `if`**.

In [95]:
X['T'] = (X['TL']+X['TM']+X['TR'] == 3)

In [96]:
X['MH'] = (X['ML']+X['MM']+X['MR'] == 3)
X['B'] = (X['BL']+X['BM']+X['BR'] == 3)
X['L'] = (X['TL']+X['ML']+X['BL'] == 3)
X['MV'] = (X['TM']+X['MM']+X['BM'] == 3)
X['R'] = (X['TR']+X['MR']+X['BR'] == 3)
X['D1'] = (X['TL']+X['MM']+X['BR'] == 3)
X['D2'] = (X['BL']+X['MM']+X['TR'] == 3)

In [97]:
for feature in ['TL','TM','TR','ML','MM','MR','BL','BM','BR']:
    X.drop(feature, axis=1, inplace=True)

In [98]:
X.columns

Index(['T', 'MH', 'B', 'L', 'MV', 'R', 'D1', 'D2'], dtype='object')

In [99]:
for feature in X.columns:
    X[feature] = X[feature].astype(int)

In [100]:
X['Final'] = np.zeros(X.shape[0])

In [101]:
for feature in X.columns:
    X['Final'] += X[feature]

In [102]:
rf2 = RandomForestClassifier(n_estimators=500, max_depth=10, random_state=19052019)
score = cross_val_score(rf2, X, y, cv=5, scoring='accuracy').mean()
print(score)

1.0


Опишите свои рассуждения в 2-3 предложениях. Почему выбранный вами метод сработал?

Мы говорим о достаточно простой игре, где можно просто напросто проверить 8 выигрышных позиций и проверять каждую игру на их наличие.